# TensorFlow + Keras con GPU (RTX 4060) en Conda + VS Code

Este cuaderno te sirve como **plantilla de verificación y arranque** para usar tu **NVIDIA RTX 4060** con:

- **Anaconda / Miniconda**
- **Python 3.10**
- **TensorFlow 2.15 (incluye Keras)**
- **VS Code + Jupyter**

> **Objetivo:** verificar que TensorFlow detecta la GPU, activar *memory growth* (para evitar errores de memoria) y entrenar un modelo pequeño de prueba.

## 1) Verificar GPU disponible desde TensorFlow

En esta celda imprimimos la versión de TensorFlow y listamos los dispositivos GPU detectados.

Si todo está bien, deberías ver algo similar a:

- `GPUs disponibles: [PhysicalDevice(name='/physical_device:GPU:0', ...)]`

In [ ]:
import tensorflow as tf

print("TensorFlow:", tf.__version__)
print("Build info (resumen):", tf.sysconfig.get_build_info().get("cuda_version", "N/A"), 
      tf.sysconfig.get_build_info().get("cudnn_version", "N/A"))

gpus = tf.config.list_physical_devices('GPU')
print("GPUs disponibles:", gpus)

## 2) Activar *Memory Growth* (recomendado)

Esto evita que TensorFlow **reserve toda la VRAM** al inicio.
Es especialmente útil si también usas navegador, VS Code, u otras apps mientras entrenas.

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ Memory growth activado para:", gpus)
    except RuntimeError as e:
        print("⚠️ RuntimeError:", e)
else:
    print("⚠️ No se detectaron GPUs. Revisa instalación/versions.")

## 3) Prueba rápida de entrenamiento (Keras)

Entrenaremos un modelo **muy pequeño** con datos sintéticos para confirmar que:
- el pipeline funciona,
- el entrenamiento se ejecuta sin errores,
- y (idealmente) se usa la GPU.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers

# Datos sintéticos (clasificación 10 clases)
n_samples = 5000
n_features = 1000
n_classes = 10

X = np.random.randn(n_samples, n_features).astype(np.float32)
y = np.random.randint(0, n_classes, size=(n_samples,))
y = keras.utils.to_categorical(y, num_classes=n_classes)

model = keras.Sequential([
    layers.Input(shape=(n_features,)),
    layers.Dense(512, activation="relu"),
    layers.Dense(256, activation="relu"),
    layers.Dense(n_classes, activation="softmax"),
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

history = model.fit(X, y, epochs=3, batch_size=64, validation_split=0.2, verbose=1)

## 4) Confirmar el dispositivo usado (opcional)

TensorFlow suele usar GPU automáticamente si está disponible.
Esta celda muestra el *placement* de operaciones para confirmar asignación de dispositivos.

In [ ]:
import tensorflow as tf

tf.debugging.set_log_device_placement(True)

# Operación pequeña para observar asignación
a = tf.random.normal((2000, 2000))
b = tf.random.normal((2000, 2000))
c = tf.matmul(a, b)

print("✅ Operación matmul ejecutada. Revisa el log de device placement arriba.")

## 5) Tips rápidos (para tu flujo con VS Code)

1. **Selecciona el kernel correcto**: `tf_gpu` (conda)  
2. Si la GPU no aparece:
   - Verifica `nvidia-smi` en terminal
   - Revisa que tu env tenga **Python 3.10**
   - Reinstala dependencias clave:
     - `conda install -c conda-forge cudatoolkit=11.8 cudnn=8.6`
     - `pip install tensorflow==2.15.0`

---

Si me dices tu **SO (Windows/Linux)** y si quieres usar **CNN con imágenes reales** (por ejemplo, MNIST o un dataset propio),
te preparo una versión del notebook ya orientada a tu caso (visión médica/UAV, etc.).